<a href="https://colab.research.google.com/github/modraqueeb/IndusVision/blob/main/Task9_MLintern.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Wed Mar 19 05:07:24 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   60C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
import os
HOME = os.getcwd()
print (HOME)

/content


In [3]:
%cd {HOME}
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1pz68D1Gsx80MoPg-_q-IbEdESEmyVLm-' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1pz68D1Gsx80MoPg-_q-IbEdESEmyVLm-" -O vehicle-counting.mp4 && rm -rf /tmp/cookies.txt

/content
--2025-03-19 05:09:44--  https://docs.google.com/uc?export=download&confirm=&id=1pz68D1Gsx80MoPg-_q-IbEdESEmyVLm-
Resolving docs.google.com (docs.google.com)... 74.125.130.100, 74.125.130.138, 74.125.130.102, ...
Connecting to docs.google.com (docs.google.com)|74.125.130.100|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1pz68D1Gsx80MoPg-_q-IbEdESEmyVLm-&export=download [following]
--2025-03-19 05:09:44--  https://drive.usercontent.google.com/download?id=1pz68D1Gsx80MoPg-_q-IbEdESEmyVLm-&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 142.251.12.132, 2404:6800:4003:c11::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|142.251.12.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 35345757 (34M) [video/mp4]
Saving to: ‘vehicle-counting.mp4’

vehicle-counting.mp 100%[===================>]  33.71M  28.1

In [4]:
SOURCE_VIDEO_PATH = f"{HOME}/vehicle-counting.mp4"

In [5]:
# Pip install method (recommended)

!pip install "ultralytics<=8.3.40"

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics 8.3.40 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 41.0/112.6 GB disk)


In [8]:
%cd {HOME}
!git clone https://github.com/ifzhang/ByteTrack.git
%cd {HOME}/ByteTrack

# Workaround for issues in the notebook
!sed -i 's/onnx==1.8.1/onnx==1.9.0/g' requirements.txt

# Install the dependencies with the correct versions
!pip3 install -q -r requirements.txt
!python3 setup.py -q develop
!pip install -q cython_bbox
!pip install -q onemetric

# Install additional dependencies
!pip install -q loguru lap thop

# Uninstall any existing YOLOX versions
!pip uninstall -y yolox

# Install the correct YOLOX version
!pip install yolox==0.8.0  # Install version 0.8.0 (or the latest available v8 version)

from IPython import display
display.clear_output()

import sys
sys.path.append(f"{HOME}/ByteTrack")

import yolox
print("yolox.__version__:", yolox.__version__)


yolox.__version__: 0.1.0


In [9]:
from yolox.tracker.byte_tracker import BYTETracker, STrack
from onemetric.cv.utils.iou import box_iou_batch
from dataclasses import dataclass


@dataclass(frozen=True)
class BYTETrackerArgs:
    track_thresh: float = 0.25
    track_buffer: int = 30
    match_thresh: float = 0.8
    aspect_ratio_thresh: float = 3.0
    min_box_area: float = 1.0
    mot20: bool = False

In [10]:
!pip install supervision==0.1.0


from IPython import display
display.clear_output()


import supervision
print("supervision.__version__:", supervision.__version__)

supervision.__version__: 0.1.0


In [11]:
from supervision.draw.color import ColorPalette
from supervision.geometry.dataclasses import Point
from supervision.video.dataclasses import VideoInfo
from supervision.video.source import get_video_frames_generator
from supervision.video.sink import VideoSink
from supervision.notebook.utils import show_frame_in_notebook
from supervision.tools.detections import Detections, BoxAnnotator
from supervision.tools.line_counter import LineCounter, LineCounterAnnotator

In [12]:
MODEL = "yolov8x.pt"

In [13]:
from ultralytics import YOLO
model= YOLO(MODEL)
model.fuse()

100%|██████████| 131M/131M [00:00<00:00, 374MB/s]


YOLOv8x summary (fused): 268 layers, 68,200,608 parameters, 0 gradients, 257.8 GFLOPs


In [14]:
%cd {HOME}
!yolo task=detect mode=predict model=yolov8x.pt conf=0.25 source={SOURCE_VIDEO_PATH}

/content
100% 131M/131M [00:00<00:00, 302MB/s]
Ultralytics 8.3.40 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLOv8x summary (fused): 268 layers, 68,200,608 parameters, 0 gradients, 257.8 GFLOPs

video 1/1 (frame 1/538) /content/vehicle-counting.mp4: 384x640 3 cars, 1 truck, 59.2ms
video 1/1 (frame 2/538) /content/vehicle-counting.mp4: 384x640 3 cars, 1 truck, 53.8ms
video 1/1 (frame 3/538) /content/vehicle-counting.mp4: 384x640 4 cars, 2 trucks, 53.7ms
video 1/1 (frame 4/538) /content/vehicle-counting.mp4: 384x640 5 cars, 1 truck, 53.8ms
video 1/1 (frame 5/538) /content/vehicle-counting.mp4: 384x640 4 cars, 1 truck, 53.8ms
video 1/1 (frame 6/538) /content/vehicle-counting.mp4: 384x640 4 cars, 1 truck, 63.1ms
video 1/1 (frame 7/538) /content/vehicle-counting.mp4: 384x640 4 cars, 1 truck, 63.1ms
video 1/1 (frame 8/538) /content/vehicle-counting.mp4: 384x640 5 cars, 1 truck, 63.1ms
video 1/1 (frame 9/538) /content/vehicle-counting.mp4: 384x640 4 cars, 1 truck, 63.1ms
